# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
print("no of rows in all the files with out header: " + str(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
empty_atrist_row = 0
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            empty_atrist_row  += 1 
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
# no of rows with empty atrist

print("no of rows with empty atrist name: " + str(empty_atrist_row))

no of rows in all the files with out header: 8056
no of rows with empty atrist name: 1236


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("no of rows along with one header " + str(sum(1 for line in f)))

no of rows along with one header 6821


## Records in csv file created matchs with all the files row count

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 

try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS eventsdb WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(" Error while creating the database" + str(e))
    

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('eventsdb')
except Exception as e:
    print("error while setting key space" + str(e))

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Extract the data from file using pandas for the queries required which will be used to validate the output of the queries when executed against the tables in cassandra

In [8]:
events_df = pd.read_csv('event_datafile_new.csv')

events_df.count()

artist           6820
firstName        6820
gender           6820
itemInSession    6820
lastName         6820
length           6820
level            6820
location         6820
sessionId        6820
song             6820
userId           6820
dtype: int64

In [9]:
events_df.query("sessionId == 338 and itemInSession == 4")

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
5810,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


In [10]:
events_df.query("userId == 10 and sessionId ==182")

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6805,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
6806,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
6807,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
6808,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [11]:
events_df[events_df['song']=='All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1313,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
1839,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5282,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95


## Functions to create table , insert rows to the tables, run the select query provided and for dropping the tables

In [12]:
## Function to create table

def create_table(session, tablename, columns_and_type, primary_key):
    """ create table with parameters passed and catch an exception and print when error occurs"""
    try:
        query_string = "CREATE TABLE "+ tablename + " (" + columns_and_type + ", PRIMARY KEY(" + primary_key + "))"
        session.execute(query_string)
    except Exception as e:
        print("Error creating table :" + tablename + " error: " + str(e))


## Funtion insert rows in to the table list from the  file provided 
## accepting tables list to reduce the reduntant code and reduce the execution time by inserting the rows to all tables  for each line in file
def insert_rows_into_table(session, filename, encoding_in, tablename_list, tables_columns_dict):
    """ read each line in the file and insert the line in to each table in the list provided"""
    with open(filename, encoding = encoding_in) as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        # loop through each line of the input file
        for line in csvreader:
            # loop through table in the list
            for tablename in tablename_list:
                try:
                    query = "INSERT INTO " +  tablename + " (" + tables_columns_dict[tablename] + ")"
                    query = query + "VALUES (" + ("%s," * len(tables_columns_dict[tablename].split(','))).rstrip(',') + ")"
 
                    ## the order in which the fields are present in the input file 
                    ## artist text, user_firstname text, user_gender varchar, item_in_session int, user_lastname text, song_length float,
                    ##user_level varchar, user_location text, session_id int, song text, user_id int                  

                    if tablename == 'events_by_session':
                        session.execute(query, ( line[0], int(line[8]), int(line[3]), line[9],float(line[5])))
                    elif tablename == 'events_by_user':
                        session.execute(query, (int(line[10]), int(line[8]), int(line[3]),  line[0], line[9], line[1], line[4] ))
                    elif tablename == 'events_by_songs':
                        session.execute(query, (line[9], int(line[10]), line[1], line[4] ))
                        
                except Exception as e:
                    print("error while executing insert for line" + str(line) + " table : " + tablename + " error :" + str(e))

## Funtion to select the data using query provided and return rows

def select_query(session, query_string):
    """ execute the query using the select statement provided """
    try:
        rows = session.execute(query_string)
        return rows
    except Exception as e:
        print("error while selecting the data error:" + str(e))
        
## Funtion to drop the tables using the tables list provided

def drop_tables(session, tablenames_list):
    """ execute the drop table for each table in the list provided"""
    for tablename in tablenames_list:
        try:
            session.execute(" DROP table " + tablename)
        except Exception as e:
            print("error while dropping the table:" + tablename + " error: " + str(e))
            

## Create tables based on the queries required

In [13]:
# Query 1 need to pull data using session id and item in session. Creating a table events_by_session with
# primary key : the data needs to be partioned on session id but as session id is not unique using item in session as part of primary key. 

# calling the fucntion create_table with proper `arguments, create_table(session, tablename, columns_and_type, primary_key)
create_table(session,'events_by_session', 'artist text, session_id int, item_in_session int,  song text, song_length float','session_id, item_in_session' )

# Query 2 need to pull data using user id and session id .  Creating a table events_by_user with 
# primary key : the data needs to be partioned on user id but as usser id is not unique using session id  and item in session to make the key unique. 
# Also the output should be sorted based on item in session using it as cluster key will be acheived

# calling the fucntion create_table with proper arguments, create_table(session, tablename, columns_and_type, primary_key)
create_table(session,'events_by_user', ' user_id int, session_id int, item_in_session int, artist text,  song text, user_firstname text,user_lastname text', '(user_id, session_id), item_in_session' )

# Query 3 need to pull data using song .  Creating a table with events_by_songs with 
# primary key : the data needs to be partioned on song and user id which makes it a unique key. 

# calling the fucntion create_table with proper arguments, create_table(session, tablename, columns_and_type, primary_key)
create_table(session,'events_by_songs', 'song text, user_id int , user_firstname text,user_lastname text' ,'song, user_id' )

## Create a list to hold all the table names created and dict to hold the columns names for the table  

In [14]:
tablenames_list = ['events_by_session', 'events_by_user' , 'events_by_songs']
tables_columns_dict = { 'events_by_session' : 'artist, session_id, item_in_session, song, song_length' , 'events_by_user' : 'user_id, session_id, item_in_session, artist,  song, user_firstname,user_lastname',
               'events_by_songs' : 'song, user_id, user_firstname, user_lastname'}

## Insert file contents to all the tables

In [15]:
insert_rows_into_table(session,'event_datafile_new.csv', 'utf8', tablenames_list, tables_columns_dict)

## Excute each query against corresponding table and see if it matches to the output extracted from dataframe (cretaed from the events file)

## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during for sessionId = 338, and itemInSession = 4

In [16]:
selected_rows1 = select_query(session,'select artist, song, song_length from events_by_session where session_id = 338 and item_in_session = 4')
resulted_query1_df = pd.DataFrame.from_records(data = selected_rows1.current_rows, columns= selected_rows1.column_names)
resulted_query1_df.head()

,artist,song,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
selected_rows2 = select_query(session,'select artist, song, user_firstname , user_lastname, item_in_session from events_by_user where user_id = 10 and session_id = 182 ')
resulted_query2_df = pd.DataFrame.from_records(data = selected_rows2.current_rows, columns= selected_rows2.column_names)
resulted_query2_df.head()

,artist,song,user_firstname,user_lastname,item_in_session
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
1,Three Drives,Greece 2000,Sylvie,Cruz,1
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
selected_rows3 = select_query(session,"select user_firstname , user_lastname from events_by_songs where song = 'All Hands Against His Own'")
resulted_query3_df = pd.DataFrame.from_records(data = selected_rows3.current_rows, columns= selected_rows3.column_names)
resulted_query3_df.head()

,user_firstname,user_lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


 ## All the quries output match with output extracted the file using pandas dataframe

### Drop the tables before closing out the sessions

In [19]:
drop_tables(session, tablenames_list)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()